In [6]:
# Import important libraries
import pandas as pd
import numpy as np

from typing import List, Optional, Tuple

In [7]:
# 1. Validation
def validate_data(
        df: pd.DataFrame,
        total_col: str = "TotalResponses",
        correct_col: str = "CorrectResponses",
        level_col: str = "Level",
        max_lives : int = 3,

) -> Tuple[pd.DataFrame, List[str]]:
    
    """
    Validates and normalises the aggregate Number Recall data so we can reconstruct trial-level rows.
    Rules assumed from your spec:
      - Level == CorrectResponses (should hold; if not, we trust CorrectResponses).
      - TotalResponses = CorrectResponses + wrong_attempts, with wrong_attempts <= max_lives.
      - Drop rows with no attempts (TotalResponses <= 0).
    """

    df = df.copy()
    notes: List[str] = []

    # Required columns present?
    for c in [total_col, correct_col, level_col]:
        if c not in df.columns:
            raise KeyError(f"Missing required column: {c}")
        
    # If Level missing, backfill from CorrectResponses
    df[level_col] = df[level_col].fillna(df[correct_col])

    # Drop rows with nulls in the key columns
    before = len(df)
    df = df.dropna(subset=[total_col, correct_col, level_col]).copy()
    dropped_nulls = before - len(df)
    if dropped_nulls > 0:
        notes.append(f"[Drop] Removed {dropped_nulls} rows with nulls in {total_col}/{correct_col}/{level_col}.")

    # Enforce Level == CorrectResponses
    mask_lvl_mismatch = df[level_col] != df[correct_col]
    if mask_lvl_mismatch.any():
        n = int(mask_lvl_mismatch.sum())
        notes.append(f"[Fix] Level != CorrectResponses for {n} rows. Overwrote Level with CorrectResponses.")
        df.loc[mask_lvl_mismatch, level_col] = df.loc[mask_lvl_mismatch, correct_col]

    # Enforce TotalResponses >= CorrectResponses
    mask_too_low = df[total_col] < df[correct_col]
    if mask_too_low.any():
        n = int(mask_too_low.sum())
        notes.append(f"[Fix] TotalResponses < CorrectResponses in {n} rows. Set TotalResponses = CorrectResponses.")
        df.loc[mask_too_low, total_col] = df.loc[mask_too_low, correct_col]

    # Enforce wrong attempts <= max_lives
    wrong_attempts = df[total_col] - df[correct_col]
    mask_too_high = wrong_attempts > max_lives
    if mask_too_high.any():
        n = int(mask_too_high.sum())
        notes.append(f"[Fix] Wrong attempts exceeded {max_lives} in {n} rows. "
                     f"Capped at CorrectResponses + {max_lives}.")
        df.loc[mask_too_high, total_col] = df.loc[mask_too_high, correct_col] + max_lives

    # Drop rows with no attempts
    before2 = len(df)
    df = df[df[total_col] > 0].copy()
    dropped_zero = before2 - len(df)
    if dropped_zero > 0:
        notes.append(f"[Drop] Removed {dropped_zero} rows with {total_col} <= 0.")

    return df

In [8]:
# 2. Reconstruct data for IRT Model ie item-response rows
def row_to_item_responses(correct, total):
    """
    Build item_id and response lists for a single aggregate row.
    - item_id = 1..total
    - first 'correct' are 1, remainder are 0

    """
    total = int(total)
    correct = int(correct)
    items = list(range(1, total + 1))
    responses = [1] * correct + [0] * max(0, total - correct)

    return items, responses

def explode_trials(
    df: pd.DataFrame,
    id_col: str = "AccountId",
    total_col: str = "TotalResponses",
    correct_col: str = "CorrectResponses",
    keep_session: bool = False

) -> pd.DataFrame:
    """
    Produce a long table with participant_id, item_id, response.

    """
    items_and_responses = df[[correct_col, total_col]].apply(
        lambda r: row_to_item_responses(r[correct_col], r[total_col]), axis=1
    )
    df = df.copy()
    df["__items"] = [ir[0] for ir in items_and_responses]
    df["__responses"] = [ir[1] for ir in items_and_responses]

    long_df = df[[id_col, "__items", "__responses"]].explode(["__items", "__responses"], ignore_index=True)
    out = long_df.rename(columns={id_col: "participant_id", "__items": "item_id", "__responses": "response"})
    return out[["participant_id", "item_id", "response"]]

In [9]:
# 3. End to end function
def transform_number_recall_to_irt(
    df: pd.DataFrame,
    *,
    id_col: str = "AccountId",
    total_col: str = "TotalResponses",
    correct_col: str = "CorrectResponses",
    level_col: str = "Level",
    max_lives: int = 3,
) -> pd.DataFrame:
    clean = validate_data(
        df,
        total_col=total_col,
        correct_col=correct_col,
        level_col=level_col,
        max_lives=max_lives,
    )
    out = explode_trials(
        clean,
        id_col=id_col,
        total_col=total_col,
        correct_col=correct_col,
    )
    return out

In [10]:
df = pd.read_excel("/Users/op24226/Desktop/PsychGames/Data/NumberRecall_UserScores.xlsx")
df.head()

,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Percentile,CreationDate,AssessmentResultId,AnswerTypeId,ResourceDescription,TotalResponses,CorrectResponses,BonusAwards,Level
0,694707,10,95,60,2000,95.238095,0,2025-10-14 10:47:41.374,6983266,2,Responses,23.0,20.0,NaN,NaN
1,694707,10,95,60,2000,95.238095,0,2025-10-14 10:47:41.374,6983266,3,Reaction Time,NaN,NaN,NaN,NaN
2,694707,10,95,60,2000,95.238095,0,2025-10-14 10:47:41.374,6983266,4,Level,NaN,NaN,3.0,19.0
3,694707,10,95,60,2000,95.238095,0,2025-10-14 10:47:41.374,6983266,6,Bonus Awards,NaN,NaN,0.0,NaN
4,694779,10,95,30,900,47.619048,0,2025-10-14 10:41:31.650,6983250,2,Responses,12.0,9.0,NaN,NaN


In [11]:
irt_df = transform_number_recall_to_irt(df)

In [12]:
irt_df.head(15)

,participant_id,item_id,response
0,694707,1,1
1,694707,2,1
2,694707,3,1
3,694707,4,1
4,694707,5,1
5,694707,6,1
6,694707,7,1
7,694707,8,1
8,694707,9,1
9,694707,10,1


In [13]:
# Change irt data to mirt format

#Get the maximum item_id for each participant
max_items = irt_df.groupby('participant_id')['item_id'].max().max()

# Create a pivot table with aggfunc to handle multiple values
wide_df = irt_df.pivot_table(
    index='participant_id',
    columns='item_id',
    values='response',
    aggfunc='first'  # Takes the first response if there are multiple
)

# Ensure all item_ids up to max are present
for i in range(1, max_items + 1):
    if i not in wide_df.columns:
        wide_df[i] = np.nan

# Sort columns numerically
wide_df = wide_df.reindex(sorted(wide_df.columns), axis=1)

wide_df.columns = [f'item_id {col}' for col in wide_df.columns]

wide_df = wide_df.reset_index()

wide_df.head()

,participant_id,item_id 1,item_id 2,item_id 3,item_id 4,item_id 5,item_id 6,item_id 7,item_id 8,item_id 9,...,item_id 74,item_id 75,item_id 76,item_id 77,item_id 78,item_id 79,item_id 80,item_id 81,item_id 82,item_id 83
0,299153,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303267,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,326141,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,326143,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,334746,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
